In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

import time

## Load Excel File

In [2]:
data = pd.read_excel('Data Set for Assignment II - 1000 Records 75 Attributes.xls')

In [3]:
data.head()

,Record Serial Number,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,...,AWALAND,APERSAUT,ABESAUT,AMOTSCO,AVRAAUT,AAANHANG,ATRACTOR,AWERKT,ABROM,OUTCOME
0,1,9,1,3,3,3,1,4,1,5,...,0,1,0,0,0,0,0,0,0,0
1,2,33,1,4,3,8,0,6,0,3,...,0,0,0,0,0,0,0,0,0,0
2,3,8,2,3,3,2,2,4,1,3,...,0,0,0,0,0,0,0,0,0,0
3,4,39,1,3,3,9,1,4,1,5,...,0,1,0,0,0,0,0,0,0,0
4,5,33,1,3,3,8,0,5,0,4,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Drop Record Serial Number
data.drop('Record Serial Number', axis=1, inplace=True)

In [5]:
data.columns

Index(['MOSTYPE ', 'MAANTHUI ', 'MGEMOMV ', 'MGEMLEEF ', 'MOSHOOFD ',
       'MGODRK ', 'MGODPR ', 'MGODOV ', 'MGODGE ', 'MRELGE ', 'MRELSA ',
       'MRELOV ', 'MFALLEEN ', 'MFGEKIND ', 'MFWEKIND ', 'MOPLHOOG ',
       'MOPLMIDD ', 'MOPLLAAG ', 'MBERHOOG ', 'MBERZELF ', 'MBERBOER ',
       'MBERMIDD ', 'MBERARBG ', 'MBERARBO ', 'MSKA ', 'MSKB1 ', 'MSKB2 ',
       'MSKC ', 'MSKD ', 'MHHUUR ', 'MHKOOP ', 'MAUT1 ', 'MAUT2 ', 'MAUT0 ',
       'MZFONDS ', 'MZPART ', 'MINKM30 ', 'MINK3045 ', 'MINK4575 ',
       'MINK7512 ', 'MINK123M ', 'MINKGEM ', 'MKOOPKLA ', 'PWAPART ',
       'PWABEDR ', 'PWALAND ', 'PPERSAUT ', 'PBESAUT ', 'PMOTSCO ', 'PVRAAUT ',
       'PAANHANG ', 'PTRACTOR ', 'PWERKT ', 'PBROM ', 'PLEVEN ', 'PPERSONG ',
       'PGEZONG ', 'PWAOREG ', 'PBRAND ', 'PZEILPL ', 'PPLEZIER ', 'PFIETS ',
       'PINBOED ', 'PBYSTAND ', 'AWAPART ', 'AWABEDR ', 'AWALAND ',
       'APERSAUT ', 'ABESAUT ', 'AMOTSCO ', 'AVRAAUT ', 'AAANHANG ',
       'ATRACTOR ', 'AWERKT ', 'ABROM ', 'OUTCOME'],

In [6]:
X_all = data.iloc[:,0:76]
y_all = data['OUTCOME']
print(X_all.shape, y_all.shape)

(1000, 76) (1000,)


## Remove Fields with Zero or 5% Variance.

In [7]:
vt = VarianceThreshold(threshold=0.05)
vt_data = vt.fit(X_all)

In [8]:
X_all = X_all.iloc[:,vt_data.get_support()]
print("Shape after removing near-zero variance fields:", X_all.shape)

Shape after removing near-zero variance fields: (1000, 60)


## Normalization

In [9]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X_all)
print("Shape of X_std: ", X_std.shape)

Shape of X_std:  (1000, 60)


## PCA

In [10]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_std)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [11]:
tls.set_credentials_file(username='abhishek.sparta', api_key='yudmTNxLLBWQrw3yNaQX')

In [12]:
eig_vals = pca.explained_variance_ratio_
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,50)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,50)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
#py.iplot(fig)

### Selecting First 37 variables

In [13]:
pca = PCA(n_components=37)
X_pca = pca.fit_transform(X_std)
print("Shape of X_pca: ", X_pca.shape)

Shape of X_pca:  (1000, 37)


## Min-Max Normalization

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [15]:
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [16]:
X_pca = scaler.fit_transform(X_pca)

# Data Split - Train, Valid and Test

In [17]:
X_all = pd.DataFrame(X_pca)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.35, random_state=123)
print("X_train:", X_train.shape, ", X_test: ", X_test.shape, ", y_train: ", y_train.shape, ", y_test: ", y_test.shape)

X_train: (650, 37) , X_test:  (350, 37) , y_train:  (650,) , y_test:  (350,)


In [19]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.427, random_state=123)
print("X_valid:", X_valid.shape, ", X_test: ", X_test.shape, ", y_valid: ", y_valid.shape, ", y_test: ", y_test.shape)

X_valid: (200, 37) , X_test:  (150, 37) , y_valid:  (200,) , y_test:  (150,)


# Machine Learning

## Cost Function

In [20]:
def cost(tn, fn, tp, fp):
    return( (fn * -1000) + (tp*9000) + (fp * -9000) )

# Keras - TF

In [21]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [22]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    y_pred = K.round(y_pred)
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

## Develop the Model

In [23]:
model = Sequential()

model.add(Dense(activation="relu", units=100, input_dim=37))
model.add(Dense(activation="sigmoid", units=1))

model.compile(loss='binary_crossentropy', 
              optimizer="adam",
              metrics=['accuracy', f1])

## Train the model

In [24]:
model.fit(np.array(X_train), np.array(y_train), epochs=200)

Epoch 1/200
650/650 [==============================] - 0s - loss: 0.4131 - acc: 0.9185 - f1: nan       
Epoch 2/200
650/650 [==============================] - 0s - loss: 0.2377 - acc: 0.9354 - f1: nan     
Epoch 3/200
650/650 [==============================] - 0s - loss: 0.2358 - acc: 0.9354 - f1: nan     
Epoch 4/200
650/650 [==============================] - 0s - loss: 0.2285 - acc: 0.9354 - f1: nan     
Epoch 5/200
650/650 [==============================] - 0s - loss: 0.2230 - acc: 0.9354 - f1: nan     
Epoch 6/200
650/650 [==============================] - ETA: 0s - loss: 0.2210 - acc: 0.9340 - f1: na - 0s - loss: 0.2178 - acc: 0.9354 - f1: nan     
Epoch 7/200
650/650 [==============================] - 0s - loss: 0.2117 - acc: 0.9354 - f1: nan     
Epoch 8/200
650/650 [==============================] - 0s - loss: 0.2053 - acc: 0.9354 - f1: nan     
Epoch 9/200
650/650 [==============================] - 0s - loss: 0.1970 - acc: 0.9354 - f1: nan     
Epoch 10/200
650/650 [==========

650/650 [==============================] - 0s - loss: 0.0020 - acc: 1.0000 - f1: nan       
Epoch 80/200
650/650 [==============================] - 0s - loss: 0.0019 - acc: 1.0000 - f1: nan        
Epoch 81/200
650/650 [==============================] - 0s - loss: 0.0019 - acc: 1.0000 - f1: nan        
Epoch 82/200
650/650 [==============================] - 0s - loss: 0.0018 - acc: 1.0000 - f1: nan        
Epoch 83/200
650/650 [==============================] - 0s - loss: 0.0018 - acc: 1.0000 - f1: nan        
Epoch 84/200
650/650 [==============================] - 0s - loss: 0.0017 - acc: 1.0000 - f1: nan       
Epoch 85/200
650/650 [==============================] - 0s - loss: 0.0017 - acc: 1.0000 - f1: nan        
Epoch 86/200
650/650 [==============================] - 0s - loss: 0.0016 - acc: 1.0000 - f1: nan        
Epoch 87/200
650/650 [==============================] - 0s - loss: 0.0015 - acc: 1.0000 - f1: nan        
Epoch 88/200
650/650 [==============================] - 0s - 

650/650 [==============================] - 0s - loss: 2.5350e-04 - acc: 1.0000 - f1: nan       
Epoch 155/200
650/650 [==============================] - 0s - loss: 2.4904e-04 - acc: 1.0000 - f1: nan        
Epoch 156/200
650/650 [==============================] - 0s - loss: 2.4280e-04 - acc: 1.0000 - f1: nan       
Epoch 157/200
650/650 [==============================] - 0s - loss: 2.3754e-04 - acc: 1.0000 - f1: nan       
Epoch 158/200
650/650 [==============================] - 0s - loss: 2.3161e-04 - acc: 1.0000 - f1: nan       
Epoch 159/200
650/650 [==============================] - 0s - loss: 2.2812e-04 - acc: 1.0000 - f1: nan       
Epoch 160/200
650/650 [==============================] - 0s - loss: 2.2501e-04 - acc: 1.0000 - f1: nan        
Epoch 161/200
650/650 [==============================] - 0s - loss: 2.2021e-04 - acc: 1.0000 - f1: nan       
Epoch 162/200
650/650 [==============================] - 0s - loss: 2.1300e-04 - acc: 1.0000 - f1: nan       
Epoch 163/200
650/650 

In [25]:
# Evaluate the model
scores = model.evaluate(np.array(X_train), np.array(y_train))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f" % (model.metrics_names[2], scores[2]))

 32/650 [>.............................] - ETA: 0s
acc: 100.00%

f1: nan


## Predict on Validation Set

In [26]:
predictions = model.predict(np.array(X_valid))
rounded = [round(x[0]) for x in predictions]
con_matrix = confusion_matrix(y_valid, rounded)

print(con_matrix)

[[193   0]
 [  0   7]]


In [27]:
cost_val = cost(con_matrix[0][0], con_matrix[0][1], con_matrix[1][1], con_matrix[1][0])
print("Cost of Validation Set: ", cost_val)

Cost of Validation Set:  63000


## Prediction on Test Set

In [28]:
predictions = model.predict(np.array(X_test))
rounded = [round(x[0]) for x in predictions]
con_matrix = confusion_matrix(y_test, rounded)

print(con_matrix)

[[142   0]
 [  0   8]]


In [29]:
cost_test = cost(con_matrix[0][0], con_matrix[0][1], con_matrix[1][1], con_matrix[1][0])
print("Cost of Test Set: ", cost_test)

Cost of Test Set:  72000
